In [ ]:
import  numpy as np
import pandas as pd
import importlib
import time
import simuls, utils 
from pycaret.regression import *
from tqdm.notebook import tqdm
importlib.reload(simuls)
importlib.reload(utils)
#ExampleThree = simuls.ExampleThree
from simuls import *
from utils import *

In [ ]:
data = ExampleThree(n = 60, sig = 1, with_test = True, seed = 1).data
data.to_df()
data.train.to_df(); data.test.to_df()
cols = [col for col in data.df.columns if "x" in col]

In [ ]:
fast_models = ["lr", "br", "llar", "svm",  "en", "gbr", "omp", "kr"] # lar
slow_models = ["et", "rf", "xgboost", "mlp", "tr", ] 

In [ ]:
res_df = pd.DataFrame({"name":fast_models}).set_index("name")
res_df[["model", "mse", "corr", "time"]] = None
res_df = pd.read_csv("Data/Ex3_1000_5.csv")

In [ ]:
tracker = []
for model in ["svm",  "en", "gbr", "omp", "kr"]:
    scores = {"mse":[], "corr":[]}
    tracker += [model]
    print("Evaluating "+ model)
    start = time.time(); n_iter = 100
    for i in tqdm(range(n_iter)):
        data = ExampleThree(n = 2000, sig=5, with_test = True, seed = i + 1).data
        data.train.to_df(); data.test.to_df()
        yt = data.test.df["y"]
        out = setup(data = data.train.df, test_data= None, target = "y", silent = True, remove_multicollinearity= True, 
            verbose = False, preprocess = False)
        tuned = tune_model(create_model(model, cross_validation = False, verbose = False), optimize = "MSE", 
            choose_better = False, verbose = False)
        preds = tuned.predict(data.test.df[cols]) 
        scores["mse"] += [mse(preds, yt)]
        scores["corr"] += [np.corrcoef(preds, yt)[0,1]]
    end = time.time() - start 
    name = get_name(tuned)
    res_df.loc[model, ["model", "mse", "corr", "time"]] = [name, np.array(scores["mse"]).mean(), np.array(scores["corr"]).mean(), end / n_iter]
    print(res_df.loc[tracker,:])
    res_df.to_csv("Data/Ex3_1000_5.csv")

    

In [ ]:
import os

In [ ]:
sdf = res_df.copy()

In [ ]:
#paths = [p for p in os.listdir("Data") if p.endswith("csv")]
paths = ['Ex3_200_1.csv','Ex3_1000_5.csv',]
#paths = ['Ex1_100_5.csv','Ex2_30_5.csv',]

In [ ]:
df = pd.read_csv("Data/Ex3_100_1.csv").set_index("model")
for path in paths:
    if path == "Ex1_30_1.csv": continue
    x = pd.read_csv("Data/"+path)
    nam = path.split(".")[0]
    x = x.set_index("model")
    x.rename({"mse":"mse_"+nam, "corr":"corr_"+nam}, inplace=True, axis = 1)
    df = df.merge(x[["mse_"+nam, "corr_"+nam]], left_index = True, right_index = True, how="left")

In [ ]:
#df.reset_index().to_csv("Data/all.csv")
#df = df.drop("name,time,corr,mse".split(","), axis =1)
df

In [ ]:
df = round(df, 4)

In [ ]:
df = df.drop(["time", "name"], axis =1).reset_index()

In [ ]:
to_latex(df)

In [ ]:
df